In [4]:
from __future__ import print_function
import torch
import numpy as np

# data

In [81]:
a = torch.tensor(np.random.randn(2,3),dtype=torch.float32)

In [82]:
print(a)

tensor([[ 0.4367,  1.8829,  0.3889],
        [-0.3218, -0.1899, -0.0228]])


In [83]:
b = torch.tensor(np.random.rand(2,3),dtype=torch.float32)

In [84]:
print(b)

tensor([[0.7290, 0.1775, 0.6117],
        [0.8579, 0.1779, 0.1370]])


In [85]:
a*b

tensor([[ 0.3183,  0.3343,  0.2379],
        [-0.2761, -0.0338, -0.0031]])

In [114]:
torch.mul?

In [87]:
torch.cat?

In [88]:
torch.cat((a,b),dim = 1)

tensor([[ 0.4367,  1.8829,  0.3889,  0.7290,  0.1775,  0.6117],
        [-0.3218, -0.1899, -0.0228,  0.8579,  0.1779,  0.1370]])

# 2 model

In [11]:
torch.nn.Embedding?

In [10]:
torch.nn.Linear?

In [52]:
torch.nn.Parameter?

In [131]:
torch.nn.ModuleDict?

In [125]:
class NET(torch.nn.Module):
    def __init__(self,num_user,\
                 num_items,num_embed,
                 hidden = None,model = 'PMF',
                 pretrain_user = None,
                 pretrain_item = None,
                 pretrain = False):
        '''
        share embedding
        '''
        super(NET,self).__init__()   
        
        ##model structure init
        ##user_embedding;
        self.model = model
        self.num_users = num_user
        self.num_items = num_items
        self.num_embed = num_embed
        
        self.user_embedding = torch.nn.Embedding(self.num_users,embedding_dim=self.num_embed)
        
        self.item_embedding = torch.nn.Embedding(self.num_items,embedding_dim=self.num_embed)
        if pretrain:
            self.user_embedding.weight = torch.nn.Parameter(torch.tensor(pretrain_user,dtype=torch.float32,requires_grad=True))
            self.item_embedding.weight = torch.nn.Parameter(torch.tensor(pretrain_item,dtype = torch.float32,requires_grad=True))
        
        ##pmf linear project 
        self.pmf_mlp = torch.nn.Linear(self.num_embed,out_features=1)
        
        ##mlp
        self.mlp = torch.nn.ModuleList()
        last_dim = 2*self.num_embed
        if hidden is not None:
            for i in hidden:
                fc = torch.nn.Linear(last_dim,i)
                last_dim = i
                self.mlp.append(fc)
        self.mlp.append(torch.nn.Linear(hidden[-1],1))
        
        self.nmf = torch.nn.Linear(hidden[-1]+self.num_embed,1)
        
    def forward(self,user_vec,item_vec):
        user_embed = self.user_embedding(user_vec)
        item_embed = self.item_embedding(item_vec)
        
        ##element product between user_embed and item_embed
        if 'PMF' == self.model:
            product_embed = user_embed*item_embed
            h1 = self.pmf_mlp(product_embed)
            
        elif 'MLP' == self.model:
            cat_embed = torch.cat((user_embed,item_embed),dim=1)
            h_in = cat_embed
            for i in range(len(self.mlp)):
                h_out = self.mlp[i](h_in)
                h_out = torch.nn.ReLU()(h_out)
                h_in = h_out
            h1 = h_out
        else:
            product_embed = user_embed*item_embed
            cat_embed = torch.cat((user_embed,item_embed),dim=1)
            h_in = cat_embed
            for i in range(len(self.mlp)-1):
                h_out = self.mlp[i](h_in)
                h_out = torch.nn.ReLU()(h_out)
                h_in = h_out
            print(product_embed)
            print(h_out)
            h1 = self.nmf(torch.cat((product_embed,h_out),dim=1))  
        output = torch.nn.Sigmoid()(h1)
        
        return output

In [126]:
net = NET(3,3,2,hidden=[3,4,5],model='NMF')

In [127]:
user = torch.tensor([1,0])
item = torch.tensor([2,2])

In [128]:
net(user,item)

tensor([[-2.1004,  0.2978],
        [-0.3253, -2.1483]], grad_fn=<MulBackward0>)
tensor([[0.0000, 0.2862, 0.1973, 0.1929, 0.2708],
        [0.0000, 0.2448, 0.1609, 0.2173, 0.2407]], grad_fn=<ReluBackward0>)


tensor([[0.6687],
        [0.5442]], grad_fn=<SigmoidBackward>)

In [129]:
net.user_embedding.weight

Parameter containing:
tensor([[ 0.1345,  1.4504],
        [ 0.8685, -0.2011],
        [ 0.1336, -0.6227]], requires_grad=True)

In [130]:
net.item_embedding.weight

Parameter containing:
tensor([[ 0.4075, -0.1322],
        [ 0.6980, -3.2352],
        [-2.4183, -1.4812]], requires_grad=True)